## Import Libraries

In [ ]:
# Standard library imports
from datetime import datetime

# Data handling and numerical processing
import pandas as pd
import numpy as np

# Geographic data handling
import geopandas as gpd
import folium
from folium.plugins import HeatMap

# Data transformation and preprocessing
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    LabelEncoder,
    MultiLabelBinarizer
)
from pyproj import Proj, transform

# Model selection and resampling
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler

# Visualization
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


## Loading Data

In [ ]:
#Load data
party_raw_a= pd.read_csv('Data raw/Ongevallengegevens/partijen.txt',dtype=str)
rta_raw_a= pd.read_csv('Data raw/Ongevallengegevens/ongevallen.txt',dtype=str)
pointlocations= pd.read_csv('Data raw/Netwerkgegevens/puntlocaties.txt')
weather_raw = pd.read_csv('Data raw/Weather/De Bilt.txt',dtype=str)



In [ ]:
#Load later transfered data
rta_raw_b=pd.read_csv('Data raw/Data transfer Rijkswaterstaat/Ongevallen.txt',delimiter=';',dtype=str)
party_raw_b=pd.read_csv('Data raw/Data transfer Rijkswaterstaat/Partijen.txt',delimiter=';',dtype=str)
partyextra_b=pd.read_csv('Data raw/Data transfer Rijkswaterstaat/Partijaanvullingen.txt',delimiter=';')
victims_b=pd.read_csv('Data raw/Data transfer Rijkswaterstaat/Slachtoffers.txt',delimiter=';',dtype=str)

## Preprocessing RTA Dataset

In [ ]:
columns_to_drop = ['DATUM_VKL', 'DAG_CODE', 'MND_NUMMER', 'JAAR_VKL', 'TIJDSTIP',
                   'UUR', 'DDL_ID', 'AP4_CODE','AP5_CODE',
                   'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH', 'ANTL_GOV',
                   'ANTL_PTJ', 'ANTL_TDT', 'MNE_CODE','MND_NUMMER']

# Drop the specified columns inplace
rta_raw_a.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Define the colorblind-friendly CUD palette with additional colors
palette = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#999999', '#F8A19F', '#6A3D9A',
           '#E6A3DC', '#B3DE3F', '#F2BC40', '#99D7E3', '#BB47B8', '#FBE64B', '#8B8B3A', '#E5AB97', '#43D0B2', '#FF69B4',
           '#8A2BE2', '#DEB887']

In [ ]:
rta_raw_b['VKL_NUMMER'] = rta_raw_b['VKL_NUMMER'].str.split(',').str[0]
rta_raw_a['VKL_NUMMER'] = rta_raw_a['VKL_NUMMER'].str.split(',').str[0]

# Convert 'VKL_NUMMER' column to integer in both dataframes
rta_raw_b['VKL_NUMMER'] = rta_raw_b['VKL_NUMMER'].astype(int)
rta_raw_a['VKL_NUMMER'] = rta_raw_a['VKL_NUMMER'].astype(int)

# Perform inner merge on 'VKL_NUMMER' column
rta_raw = pd.merge(rta_raw_b, rta_raw_a, on='VKL_NUMMER', how='inner')


In [ ]:
#Maybe not drop
rta_filtered=rta_raw



In [ ]:
# Convert DATUM_VKL to datetime format
rta_filtered['DATUM_VKL'] = pd.to_datetime(rta_filtered['DATUM_VKL'], format='%Y%m%d')

# Extract year, month, and weekday
rta_filtered['YEAR'] = rta_filtered['DATUM_VKL'].dt.year
rta_filtered['MONTH'] = rta_filtered['DATUM_VKL'].dt.month
rta_filtered['WEEKDAY'] = rta_filtered['DATUM_VKL'].dt.weekday

# Create a new column indicating weekend (1 for weekend, 0 for weekday)
rta_filtered['WEEKEND'] = rta_filtered['WEEKDAY'].apply(lambda x: 1 if x >= 5 else 0)


In [ ]:
def concatenate_columns(df, col_list, new_col_name):
    # Replace empty strings with NaN
    df[col_list] = df[col_list].replace('', np.nan)

    # Concatenate the columns, replace NaN values with an empty string and join the non-empty values with a semicolon
    df[new_col_name] = df[col_list].apply(lambda row: ';'.join(row.dropna()), axis=1)

# Define the columns to concatenate for BZD_VM
columns_to_concat = ['BZD_ID_VM1', 'BZD_ID_VM2', 'BZD_ID_VM3']
concatenate_columns(rta_filtered, columns_to_concat, 'BZD_VM')

# Repeat the same process for BZD_IF
columns_to_concat = ['BZD_ID_IF1', 'BZD_ID_IF2', 'BZD_ID_IF3']
concatenate_columns(rta_filtered, columns_to_concat, 'BZD_IF')

# Repeat the same process for BZD_TA
columns_to_concat = ['BZD_ID_TA1', 'BZD_ID_TA2', 'BZD_ID_TA3']
concatenate_columns(rta_filtered, columns_to_concat, 'BZD_TA')

In [ ]:
palette = {
    "dark_blue": "#003366",
    "orange_brown": "#cc9933",
    "green": "#339900",
}

highlight_color = "#008EC6"

In [ ]:
category_mapping = {'DOD': 1, 'LZW': 1, 'LLI': 0, 'UMS': 0}
rta_filtered['SEVERE'] = rta_filtered['AP4_CODE'].map(category_mapping)

# Group by AP4_CODE and SEVERE and count occurrences
code_severity_counts = rta_filtered.groupby(['AP4_CODE', 'SEVERE']).size().unstack(fill_value=0)

# Sorting the DataFrame according to a specific order
order = ['UMS', 'LLI', 'LZW', 'DOD']
code_severity_counts = code_severity_counts.reindex(order)

# Define colors for the severity levels
palette = {"dark_blue": "#003366", "orange_brown": "#cc9933", "green": "#339900"}
colors = [palette['dark_blue'], palette['orange_brown']]  # Using dark blue for non-severe, orange brown for severe

# Plotting configuration
code_severity_counts.plot(kind='bar', stacked=True, color=colors)
plt.title('Counts of AP4_CODE by Severity')
plt.xlabel('AP4_CODE')
plt.ylabel('Counts')
plt.xticks(rotation=0)  # Keep labels horizontal
plt.legend(title='Severity', labels=['Non-severe', 'Severe'])
plt.show()

In [ ]:
# Define the colorblind-friendly CUD palette with additional colors
palette = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#999999', '#F8A19F', '#6A3D9A',
           '#E6A3DC', '#B3DE3F', '#F2BC40', '#99D7E3', '#BB47B8', '#FBE64B', '#8B8B3A', '#E5AB97', '#43D0B2', '#FF69B4',
           '#8A2BE2', '#DEB887']

## for the data description use the final dataset and the orignal, plot them here

In [ ]:
remove_columns=['BZD_ID_VM1', 'BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN',
                      'BZD_ID_IF1','BZD_ID_IF2', 'BZD_ID_IF3', 'BZD_IF_AN',
                      'BZD_ID_TA1', 'BZD_ID_TA2','BZD_ID_TA3', 'BZD_TA_AN',
                      'WDK_AN','WSE_AN','WVG_AN','HUISNUMMER',
                      'GME_NAAM', 'PVE_NAAM', 'KDD_NAAM', 'PLT_NAAM',
                      'DIENSTCODE', 'DIENSTNAAM', 'DISTRCODE', 'DISTRNAAM',
                      'DAGTYPE','WEEKNR','REGNUMMER', 'PVOPGEM', 'AP5_CODE',
                      'ZAD_ID','WGD_CODE_2','HECTOMETER','JTE_ID','WVK_ID','AP3_CODE',
                      'AP4_CODE', 'ANTL_DOD','ANTL_SLA','ANTL_GOV', 'ANTL_GZH',
                      'ANTL_SEH','NIVEAUKOP']
# Calculating the missing value percentage for each column before dropping them
missing_percentage_before_drop = rta_filtered[remove_columns].isnull().mean() * 100

missing_percentage_before_drop = missing_percentage_before_drop.to_frame(name='Missing Value Percentage')
missing_percentage_before_drop.reset_index(inplace=True)
missing_percentage_before_drop.columns = ['Column', 'Missing Value Percentage']


print(missing_percentage_before_drop)

In [ ]:
rta_filtered['ANTL_DOD'].value_counts()

In [ ]:
rta_filtered.drop(columns=remove_columns,inplace=True)

rta=rta_filtered

In [ ]:
# Add coordinates to rta dataset
rta=pd.merge(rta_filtered,pointlocations,on='FK_VELD5')

In [ ]:
rta_raw['GME_ID'].nunique()

In [ ]:

from pyproj import CRS, Transformer

# Function to convert RD to GPS
def rd_to_gps(x, y):
    # Define the coordinate systems using CRS objects
    rd_crs = CRS('EPSG:28992')  # RD New
    wgs84_crs = CRS('EPSG:4326')  # WGS 84

    # Create a Transformer object to perform the conversion
    transformer = Transformer.from_crs(rd_crs, wgs84_crs, always_xy=True)

    # Perform the transformation
    lon, lat = transformer.transform(x, y)
    return lat, lon


# Apply the conversion function
rta['latitude'], rta['longitude'] = zip(*rta.apply(lambda row: rd_to_gps(row['X_COORD'], row['Y_COORD']), axis=1))

# Create a new dataframe with the selected columns
coordinates = rta[['VKL_NUMMER', 'SEVERE', 'latitude', 'longitude']]

# Save the result to a new CSV file
coordinates.to_csv('converted_coordinates.csv', index=False)

print("Conversion and saving complete!")

In [ ]:
coordinates = pd.read_csv('converted_coordinates.csv')
coordinates.drop('SEVERE',axis=1,inplace=True)
rtacoord=pd.merge(rta,coordinates,on='VKL_NUMMER')


In [ ]:
coordinates.columns

In [ ]:
rta= rtacoord

In [ ]:
rta.head()

## SEVERE Only

In [ ]:

# Filter data to include only coordinates within the boundaries of the Netherlands
netherlands = df[(df['latitude'] >= 50) & (df['latitude'] <= 54) & (df['longitude'] >= 3) & (df['longitude'] <= 8)]

# Map severity values
severity_map = {0: 'Non-Severe', 1: 'Severe'}
netherlands['SEVERITY_LABEL'] = netherlands['SEVERE'].map(severity_map)

# Get unique categories
severities = netherlands['SEVERITY_LABEL'].unique()
types = netherlands['TYPE'].unique()

# Create subplots
fig, axes = plt.subplots(len(severities), len(types), figsize=(15, 10), sharex=True, sharey=True)

for i, severity in enumerate(severities):
    for j, type_ in enumerate(types):
        subset = netherlands[(netherlands['SEVERITY_LABEL'] == severity) & (netherlands['TYPE'] == type_)]
        if subset.empty:
            axes[i, j].set_visible(False)
        else:
            hb = axes[i, j].hexbin(subset['longitude'], subset['latitude'], gridsize=250, cmap='viridis', norm=LogNorm())
            axes[i, j].set_title(f'{type_},{severity}', fontsize=22)
            if i == len(severities) - 1:
                axes[i, j].set_xlabel('Longitude', fontsize=20)
            if j == 0:
                axes[i, j].set_ylabel('Latitude', fontsize=20)
            cbar = fig.colorbar(hb, ax=axes[i, j])
            cbar.ax.tick_params(labelsize=16)

# Adjust font sizes for tick labels
for ax in axes.flat:
    if ax.get_visible():
        ax.tick_params(axis='both', which='major', labelsize=16)

plt.tight_layout()
plt.show()

In [ ]:
rta.columns

In [ ]:
#Lastig de verkeersintensiteit mee te pakken
# Ik neem coronajaren mee als feture, want ik denk niet dat covid wel impact heeft op de hoveelheid ongelukken maar niet op de severity. Afweging meer data prioriteit
# After the outbreak of COVID-19, aggressiveness is estimated to increase by 0.023 and inattentiveness is estimated to increase by 0.015, resulting in a total indirect effect of 0.205 (3.598 × 0.023 + 8.123 × 0.015) on severity propensity. However, according to the probit model in Table 4, COVID-19 is not found to affect crash severity significantly, likely because risky driving behaviors (e.g., speeding, distraction) that COVID-19 is associated with are used to model crash severity directly.
# Unlike the SEM assumption that COVID-19 affected crash severity via changing driving behaviors, the probit model assumes COVID-19 imposed a direct effect on crash severity. Estimation results of the probit model are reported in Table 4 . All the explanatory variables were regarded as statistically significant at 95% level (p-values < 0.05) in the proposed SEM, whereas the variables COVID-19 and distraction were found to be insignificant in the probit model for the crash severity.
# heeft impact, maar wellicht niet al te significant en wordt met jaar al meegepakt.
#imapct of covid on crash severity. Heeft wel impact, maar de mate verschillend. Binary zou niet jjustice doen en data is schaars dus ik doe het gewoon per jaar, na covid is de geiddelde snelheid vgm ook hoger
# How did COVID-19 impact driving behaviors and crash Severity? A multigroup structural equation modeling
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9042805/

# Meer alcohol en drugs cases o.a. behaviour is dus anders, maar ook na covid. Ik dnek dat jaar dit het beste kan oppakken, eerder dan een binary variable
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9874053/
rta['DATUM_VKL'] = pd.to_datetime(rta['DATUM_VKL'])

# Define the start and end dates
#start_date = pd.to_datetime('2020-03-01')
#end_date = pd.to_datetime('2021-05-01')


# Filter out rows falling within the specified date range


# Convert 'DATUM_VKL' column back to the original format
rta['DATUM_VKL'] = rta['DATUM_VKL'].dt.strftime('%Y%m%d')

In [ ]:
print(rta.shape)
print(rta['SEVERE'].value_counts())

In [ ]:
rta.to_csv('Processed data/rta.csv',index=True)

## Weather


In [ ]:
weather_raw['YYYYMMDD']=weather_raw['YYYYMMDD'].astype(int)

# Filter the datetime between 2013 and 2022
weather_filtered = weather_raw[(weather_raw['YYYYMMDD'] >= 20130101) & (weather_raw['YYYYMMDD'] <= 20221231)]

# Remove leading spaces from column names
weather_filtered.columns = weather_filtered.columns.str.strip()


In [ ]:
numeric_columns = ['FG', 'FHX', 'FHN', 'FXX', 'TG', 'TN', 'TX', 'SQ', 'DR', 'RH', 'PG', 'VVN', 'VVX', 'NG']

# Convert columns to numeric safely using .loc to avoid SettingWithCopyWarning
for column in numeric_columns:
    weather_filtered.loc[:, column] = pd.to_numeric(weather_filtered.loc[:, column], errors='coerce')



In [ ]:
# Handling special cases for precipitation where -1 indicates values below measurable limits --> 0.01 to show there is something rather then nothing, but very small
weather_filtered = weather_filtered.copy()  # Create a copy of the DataFrame
weather_filtered['RH'] = weather_filtered['RH'].apply(lambda x: 0.01 if x == -1 else x)
weather_filtered['RHX'] = weather_filtered['RHX'].apply(lambda x: 0.01 if x == -1 else x)

# Selecting specific columns and making a copy for further processing
weather_processed = weather_filtered[['YYYYMMDD', 'FG', 'FHX', 'FHN', 'FXX', 'TG', 'TN', 'TX', 'SQ', 'DR', 'RH', 'PG', 'VVN', 'VVX', 'NG']].copy()
weather_processed.columns = [
    'DATUM_VKL',
    'MeanWindSpeed_mps',
    'MaxHourlyWindSpeed_mps',
    'MinHourlyWindSpeed_mps',
    'MaxWindGust_mps',
    'MeanTemperature_C',
    'MinTemperature_C',
    'MaxTemperature_C',
    'SunshineDuration_hrs',
    'PrecipitationDuration_hrs',
    'TotalDailyPrecip_mm',
    'MeanSLPressure_hPa',
    'MinVisibility_km',
    'MaxVisibility_km',
    'MeanCloudCover_oct'
]

# Define columns to convert and apply conversion factor
cols_to_convert = [
    'MeanWindSpeed_mps', 'MaxHourlyWindSpeed_mps', 'MinHourlyWindSpeed_mps',
    'MaxWindGust_mps', 'MeanTemperature_C', 'MinTemperature_C',
    'MaxTemperature_C', 'SunshineDuration_hrs', 'PrecipitationDuration_hrs',
    'TotalDailyPrecip_mm', 'MeanSLPressure_hPa'
]

# Convert selected columns to numeric types, apply conversion factor, and handle errors
for col in cols_to_convert:
    weather_processed[col] = pd.to_numeric(weather_processed[col], errors='coerce') * 0.1


In [ ]:
weather_processed['DATUM_VKL']=weather_processed['DATUM_VKL'].astype(str)

In [ ]:
rta= pd.merge(rta,weather_processed,on='DATUM_VKL',how='left')

## Parties involved CHECKPOINT!

In [ ]:

party_merged= pd.merge(party_raw_a,party_raw_b,on='PTJ_ID')
party_merged = party_merged.drop(columns=[col for col in party_merged.columns if col.endswith('_x')])
party_merged.columns = [col.replace('_y', '') if '_y' in col else col for col in party_merged.columns]

In [ ]:
empty_cols = [col for col in party_merged.columns if party_merged[col].isna().all()]
party_merged.drop(columns=empty_cols, inplace=True)

In [ ]:
party_merged.columns

Meaning that LIKE_ID and Geslacht are often missing together, but not too often with objectype, so  age could be inputed  by mode of objet type. Unfortunately not that many other columns to use for imputing or using KNN, as most columns are almsot empty.


In [ ]:
party_merged['VKL_NUMMER'] = party_merged['VKL_NUMMER'].str.split(',').str[0].astype(int)

In [ ]:
party_merged = party_merged.merge(rta[['VKL_NUMMER', 'SEVERE']], on='VKL_NUMMER', how='left')

In [ ]:
party_merged.columns

In [ ]:
import seaborn as sns
from matplotlib.colors import LogNorm

# Example data processing setup
# Calculate NaN percentages for each VKL_NUMMER for each 'SEVERE' category
nan_percentages = party_merged.groupby(['VKL_NUMMER', 'SEVERE']).apply(
    lambda df: pd.Series({
        'OTE_ID_NaN_Percentage': df['OTE_ID'].isna().mean() * 100,
        'LKE_ID_NaN_Percentage': df['LKE_ID'].isna().mean() * 100
    })).reset_index()

# Binning the NaN percentages into 10 equal bins
nan_percentages['OTE_ID_Bin'] = pd.cut(nan_percentages['OTE_ID_NaN_Percentage'], bins=10, labels=False, include_lowest=True)
nan_percentages['LKE_ID_Bin'] = pd.cut(nan_percentages['LKE_ID_NaN_Percentage'], bins=10, labels=False, include_lowest=True)

# Create separate pivot tables for SEVERE=0 and SEVERE=1
heatmap_data_severe_0 = nan_percentages[nan_percentages['SEVERE'] == 0].pivot_table(
    index='LKE_ID_Bin', columns='OTE_ID_Bin', values='VKL_NUMMER', aggfunc='count', fill_value=0)

heatmap_data_severe_1 = nan_percentages[nan_percentages['SEVERE'] == 1].pivot_table(
    index='LKE_ID_Bin', columns='OTE_ID_Bin', values='VKL_NUMMER', aggfunc='count', fill_value=0)

# Using subplots to plot the heatmaps
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 8))

# Heatmap for SEVERE=0
sns.heatmap(heatmap_data_severe_0, annot=True, fmt="d", cmap='Blues', norm=LogNorm(vmin=1, vmax=heatmap_data_severe_0.max().max()), ax=axes[0])
axes[0].set_title('Heatmap of VKL_NUMMER Count by NaN Percentage Bins for SEVERE=0')
axes[0].set_xlabel('OTE_ID NaN Percentage Bins')
axes[0].set_ylabel('LKE_ID NaN Percentage Bins')
axes[0].invert_yaxis()  # Invert the y-axis
axes[0].tick_params(axis='y', which='both', labelleft=True)  # Ensure y-axis labels are visible
axes[0].get_xaxis().set_visible(False)  # Hide x-axis labels

# Heatmap for SEVERE=1
sns.heatmap(heatmap_data_severe_1, annot=True, fmt="d", cmap='Blues', norm=LogNorm(vmin=1, vmax=heatmap_data_severe_1.max().max()), ax=axes[1])
axes[1].set_title('Heatmap of VKL_NUMMER Count by NaN Percentage Bins for SEVERE=1')
axes[1].set_xlabel('OTE_ID NaN Percentage Bins')
axes[1].set_ylabel('')  # Remove y-axis label
axes[1].invert_yaxis()  # Invert the y-axis
axes[1].tick_params(axis='y', which='both', labelleft=True)  # Ensure y-axis labels are visible
axes[1].get_xaxis().set_visible(False)  # Hide x-axis labels

plt.tight_layout()
plt.show()

In [ ]:
nan_counts = party_merged.isna().sum()
print(nan_counts)

In [ ]:
# Counting non-NaN values in each column
non_nan_counts = party_merged.count()

# Displaying the counts
print(non_nan_counts)


In [ ]:
# Grouping the data and creating the matrix
matrix = party_merged.groupby(['OTE_ID', 'LKE_ID'], dropna=False).size().unstack(fill_value=0)

# Setting up the matplotlib figure
plt.figure(figsize=(12, 10))

# Defining the logarithmic norm. We add 1 to the matrix values to handle zero values because log(0) is undefined.
log_norm = LogNorm(vmin=1, vmax=matrix.max().max())

# Choosing a colormap that provides good contrast in the log scale
color_scale = sns.diverging_palette(220, 20, as_cmap=True)

# Drawing the heatmap
ax = sns.heatmap(matrix, annot=True, fmt="d", cmap='Blues', norm=log_norm, cbar_kws={'label': 'Log-scaled Count of Records'})

# Adding title and formatting
plt.title('Logarithmic Scaled Heatmap of Record Counts by OTE_ID and LKE_ID')
plt.xlabel('LKE_ID')  # Setting the label for the x-axis
plt.ylabel('OTE_ID')  # Setting the label for the y-axis

# Optimizing the layout for better viewing
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
# Group the data by 'OTE_ID' and calculate the percentage of missing values for each group
missing_percentage_by_ote_id = (party_merged.groupby('OTE_ID')['LKE_ID']
                                .apply(lambda x: (x.isna().sum() / len(x)) * 100)
                                .rename('Percentage Missing')
                                .reset_index())

# Display the result
print(missing_percentage_by_ote_id)


In [ ]:
# Count VKL_NUMMER entries before removal
total_entries = len(party_merged['VKL_NUMMER'].unique())
severe_0_entries_before = len(party_merged[(party_merged['SEVERE'] == 0)]['VKL_NUMMER'].unique())
severe_1_entries_before = len(party_merged[(party_merged['SEVERE'] == 1)]['VKL_NUMMER'].unique())

# Remove VKL_NUMMER with 100% NaN values in either OTE_ID or LKE_ID
nan_vkl_numbers = nan_percentages[nan_percentages['OTE_ID_NaN_Percentage'] == 100]['VKL_NUMMER'].unique()
nan_vkl_numbers = set(nan_vkl_numbers).union(set(nan_percentages[nan_percentages['LKE_ID_NaN_Percentage'] == 100]['VKL_NUMMER'].unique()))
party_merged = party_merged[~party_merged['VKL_NUMMER'].isin(nan_vkl_numbers)]

# Count VKL_NUMMER entries after removal
total_entries_after = len(party_merged['VKL_NUMMER'].unique())
severe_0_entries_after = len(party_merged[(party_merged['SEVERE'] == 0)]['VKL_NUMMER'].unique())
severe_1_entries_after = len(party_merged[(party_merged['SEVERE'] == 1)]['VKL_NUMMER'].unique())

print(f"Total VKL_NUMMER entries deleted: {total_entries - total_entries_after}")
print(f"Deleted VKL_NUMMER entries with SEVERE=0: {severe_0_entries_before - severe_0_entries_after}")
print(f"Deleted VKL_NUMMER entries with SEVERE=1: {severe_1_entries_before - severe_1_entries_after}")


In [ ]:
party_merged_filtered=party_merged

In [ ]:
# Save the DataFrame to a CSV file
party_merged_filtered.to_csv('Processed data/party_merged_filtered.csv', index=False)


## Checkpoint!

In [ ]:
party = pd.read_csv('Processed data/party_merged_filtered.csv')
objecttypes_grouped= pd.read_csv('Data raw/ReferentiebestandenOngevallen/ObjecttypesV2.csv', delimiter=';')

In [ ]:
party['OTE_ID']=party['OTE_ID'].astype('float64')

In [ ]:
party_obj=pd.merge(party,objecttypes_grouped,on='OTE_ID')

In [ ]:
party_obj.drop(columns=['OTE_OMS','OTE_ID'],axis=1, inplace=True)

In [ ]:
import pandas as pd

value_counts_OTE_ID = party_obj['Group'].value_counts(dropna=False)
value_counts_RIJBEWGEL = party_obj['RIJBEWGEL'].value_counts(dropna=False)
value_counts_RIJBEWBEG = party_obj['RIJBEWBEG'].value_counts(dropna=False)
value_counts_TDT_ID_1 = party_obj['TDT_ID_1'].value_counts(dropna=False)

print("Value counts for RIJBEWGEL, including NaNs:")
print(value_counts_RIJBEWGEL)
print("\nValue counts for RIJBEWBEG, including NaNs:")
print(value_counts_RIJBEWBEG)
print("\nValue counts for TDT_ID_1, including NaNs:")
print(value_counts_TDT_ID_1)

print(value_counts_OTE_ID)

In [ ]:
party_obj['PTJ_ID_count'] = party_obj.groupby('VKL_NUMMER').apply(
    lambda x: x.loc[x['Group'].notna() & (x['Group'] != 'Object'), 'PTJ_ID'].count()
).reset_index(level=0, drop=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
party_obj['PTJ_ID_count'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Log-Scale Bar Plot of PTJ_ID_count')
plt.xlabel('PTJ_ID_count')
plt.ylabel('Frequency (log scale)')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np

# Calculate z-scores for PTJ_ID_count
z_scores = np.abs((party_obj['PTJ_ID_count'] - party_obj['PTJ_ID_count'].mean()) / party_obj['PTJ_ID_count'].std())

# Find PTJ_ID_count values for which the z-score is above 3
outliers = party_obj[z_scores > 3]['PTJ_ID_count'].unique()

# Count the number of 'SEVERE'=1 occurrences for outliers
severe_1_count_for_outliers = party_obj[(party_obj['SEVERE'] == 1) & party_obj['PTJ_ID_count'].isin(outliers)]['SEVERE'].sum()

# Count the number of 'SEVERE'=1 occurrences for all data
total_severe_1_count = party_obj[party_obj['SEVERE'] == 1]['SEVERE'].sum()

print("PTJ_ID_count values with z-score above 3:", outliers)
print("Number of 'SEVERE'=1 occurrences that would be deleted if outliers are removed:", severe_1_count_for_outliers)
print("Total number of 'SEVERE'=1 occurrences in the dataset:", total_severe_1_count)


In [ ]:
import numpy as np

# Calculate z-scores for PTJ_ID_count
z_scores = np.abs((party_obj['PTJ_ID_count'] - party_obj['PTJ_ID_count'].mean()) / party_obj['PTJ_ID_count'].std())

# Find PTJ_ID_count values for which the z-score is above 3
outliers = party_obj[z_scores > 3]['PTJ_ID_count'].unique()

# Group the data by VKL_NUMMER and count the number of 'SEVERE'=1 occurrences
severe_1_count_per_vkl_nummer = party_obj[party_obj['SEVERE'] == 1].groupby('VKL_NUMMER')['SEVERE'].sum()

# Filter the counts for VKL_NUMMER values that are outliers
severe_1_count_for_outliers = severe_1_count_per_vkl_nummer[severe_1_count_per_vkl_nummer.index.isin(outliers)].sum()

# Count the total number of 'SEVERE'=1 occurrences in the dataset
total_severe_1_count = party_obj[party_obj['SEVERE'] == 1]['SEVERE'].sum()

print("Number of 'SEVERE'=1 occurrences that would be deleted if outliers are removed:", severe_1_count_for_outliers)
print("Total number of 'SEVERE'=1 occurrences in the dataset:", total_severe_1_count)


In [ ]:
severity_distribution = party_obj.groupby('PTJ_ID_count')['SEVERE'].unique()

print(severity_distribution)

In [ ]:
# Calculate z-scores for PTJ_ID_count
z_scores = np.abs((party_obj['PTJ_ID_count'] - party_obj['PTJ_ID_count'].mean()) / party_obj['PTJ_ID_count'].std())

# Find PTJ_ID_count values for which the z-score is above 3
outliers = party_obj[z_scores > 3]['PTJ_ID_count'].unique()

# Filter the dataset to remove outliers
party_obj_no_outliers = party_obj[~party_obj['PTJ_ID_count'].isin(outliers)]

# Count the number of 'SEVERE'=1 occurrences in the original dataset
total_severe_1_count_original = party_obj[party_obj['SEVERE'] == 1]['SEVERE'].sum()

# Count the number of 'SEVERE'=1 occurrences in the dataset without outliers
total_severe_1_count_no_outliers = party_obj_no_outliers[party_obj_no_outliers['SEVERE'] == 1]['SEVERE'].sum()

# Calculate the difference in 'SEVERE'=1 occurrences between the original dataset and the dataset without outliers
severe_1_deleted_with_outliers_removed = total_severe_1_count_original - total_severe_1_count_no_outliers

print("Number of 'SEVERE'=1 occurrences that would be deleted if outliers are removed:", severe_1_deleted_with_outliers_removed)


In [ ]:
# Filter the dataset to include only rows where SEVERE=1
severe_1_data = party_obj[party_obj['SEVERE'] == 1]

# Count the number of unique VKL_NUMMER values with SEVERE=1 in the original dataset
unique_vkl_nummers_original = severe_1_data['VKL_NUMMER'].nunique()

# Filter the dataset to remove outliers in PTJ_ID_count
party_obj_no_outliers = party_obj[~party_obj['PTJ_ID_count'].isin(outliers)]

# Filter the dataset without outliers to include only rows where SEVERE=1
severe_1_data_no_outliers = party_obj_no_outliers[party_obj_no_outliers['SEVERE'] == 1]

# Count the number of unique VKL_NUMMER values with SEVERE=1 in the dataset without outliers
unique_vkl_nummers_no_outliers = severe_1_data_no_outliers['VKL_NUMMER'].nunique()

# Calculate the difference in unique VKL_NUMMER counts with SEVERE=1 between the original dataset and the dataset without outliers
lost_vkl_nummers_with_outliers_removed = unique_vkl_nummers_original - unique_vkl_nummers_no_outliers

print("Number of VKL_NUMMERS with SEVERE=1 that would be lost if outliers are removed:", lost_vkl_nummers_with_outliers_removed)
# Calculate the total number of rows represented by VKL_NUMMER values with SEVERE=1 in the original dataset
total_rows_severe_1_original = severe_1_data.shape[0]

# Calculate the total number of unique VKL_NUMMER values with SEVERE=1 in the original dataset
unique_vkl_nummers_severe_1_original = severe_1_data['VKL_NUMMER'].nunique()

# Calculate the total number of rows represented by VKL_NUMMER values with SEVERE=1 that would be lost if outliers are removed
total_rows_severe_1_no_outliers = severe_1_data_no_outliers.shape[0]

# Calculate the total number of unique VKL_NUMMER values with SEVERE=1 that would be lost if outliers are removed
unique_vkl_nummers_severe_1_no_outliers = severe_1_data_no_outliers['VKL_NUMMER'].nunique()

# Calculate the loss in VKL_NUMMER for which SEVERE=1 and the total rows of VKL_NUMMERS if outliers are removed
lost_vkl_nummers_severe_1 = unique_vkl_nummers_severe_1_original - unique_vkl_nummers_severe_1_no_outliers
lost_total_rows_severe_1 = total_rows_severe_1_original - total_rows_severe_1_no_outliers

print("Loss in VKL_NUMMER for which SEVERE=1 if outliers are removed:", lost_vkl_nummers_severe_1)
print("Loss in total rows of VKL_NUMMERS if outliers are removed:", lost_total_rows_severe_1)


# Calculate the total number of rows with SEVERE=1 in the original dataset
total_severe_1_original = party_obj[party_obj['SEVERE'] == 1].shape[0]

# Calculate the total number of unique VKL_NUMMER values with SEVERE=1 in the original dataset
unique_vkl_nummers_severe_1_original = severe_1_data['VKL_NUMMER'].nunique()

# Calculate the percentage of SEVERE=1 occurrences that would be lost if outliers are removed
percentage_severe_1_lost = (lost_total_rows_severe_1 / total_severe_1_original) * 100

# Calculate the percentage of unique VKL_NUMMER values with SEVERE=1 that would be lost if outliers are removed
percentage_vkl_nummers_severe_1_lost = (lost_vkl_nummers_severe_1 / unique_vkl_nummers_severe_1_original) * 100

print("Percentage of SEVERE=1 occurrences lost if outliers are removed:", percentage_severe_1_lost)
print("Percentage of unique VKL_NUMMER values with SEVERE=1 lost if outliers are removed:", percentage_vkl_nummers_severe_1_lost)


In [ ]:
# Calculate z-scores for PTJ_ID_count
z_scores = np.abs((party_obj['PTJ_ID_count'] - party_obj['PTJ_ID_count'].mean()) / party_obj['PTJ_ID_count'].std())

# Find PTJ_ID_count values for which the z-score is above 3
outliers = party_obj[z_scores > 3]['PTJ_ID_count'].unique()

# Count the number of VKL_NUMMERS and 'SEVERE'=1 for each frequency of PTJ_ID_count values
counts_per_frequency = party_obj.groupby('PTJ_ID_count').agg({'VKL_NUMMER': 'count', 'SEVERE': lambda x: (x == 1).sum()}).reset_index()
counts_per_frequency.rename(columns={'VKL_NUMMER': 'VKL_NUMMER_Count', 'SEVERE': 'SEVERE=1_Count'}, inplace=True)

# Calculate percentage of 'SEVERE'=1 occurrences
counts_per_frequency['SEVERE=1_Percentage'] = (counts_per_frequency['SEVERE=1_Count'] / counts_per_frequency['VKL_NUMMER_Count']) * 100

print("PTJ_ID_count values with z-score above 3:", outliers)
print("Number of VKL_NUMMERS, 'SEVERE'=1 count, and percentage of 'SEVERE'=1 per frequency of PTJ_ID_count values:")
print(counts_per_frequency)


Niet verwijderen wordt toch geaggregeert naar ratios gezien dat absolute aantallen information leakage kunnen zijn

New zeeland paper had de dezelfde verdeling

In [ ]:
party_obj['Party_cat'] = party_obj['PTJ_ID_count'].apply(lambda x: 'single-party' if x == 1 else ('two-party' if x == 2 else 'multiparty'))


In [ ]:
party_obj.head()

In [ ]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [ ]:
party_obj['BWG_ID_1'].value_counts(dropna=False)

In [ ]:
party_obj['VTGVERZ'].value_counts(dropna=False)

In [ ]:
#Only yes
party_obj['RIJBEWGEL'].value_counts(dropna=False)

In [ ]:
#Could be interesting but so many empty values i think it is not worth it (better to use different dataset for this type of research)
party_obj['AGT']= party_obj['AGT_ID_1'].map({11: 'Voor', 12: 'Voor', 13: 'Voor', 14: 'Achter', 15: 'Achter', 16: 'Achter', 17: 'Achter', 18: 'Flank'})
party_obj['AGT'].value_counts(dropna=False)


In [ ]:
#Only yes for 3k values, not the dataset to research this
party_obj['RIJBEWBEG'].value_counts(dropna=False)

In [ ]:
#unfortunately to empty to use, to research this aswell better to use a different set
party_obj['TDT_ID_1'].value_counts(dropna=False)

In [ ]:
# Function to calculate gender ratio
def gender_ratio(series):
    counts = series.value_counts(normalize=True)
    return counts.to_dict()

# Custom function to safely extract the mode
def safe_mode(series):
    mode_values = series.mode()
    if not mode_values.empty:
        return mode_values[0]  # Return the first mode value if exists
    else:
        return None  # Return None if the mode cannot be determined

# Aggregation functions
aggregations = {
    'GESLACHT': gender_ratio,
    'Party_cat': 'first',
    'LKE_ID': ['min', 'max', safe_mode],
    'Group': lambda x: ', '.join(x.dropna().astype(str).unique())
}

# Group by 'VKL_NUMMER' and apply aggregation
grouped = party_obj.groupby('VKL_NUMMER').agg(aggregations)

# Flatten the column names
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

# Rename columns to make them more readable
grouped.rename(columns={
    'GESLACHT_gender_ratio': 'Gender Ratio',
    'Party_cat_first': 'Party Category',
    'LKE_ID_min': 'Min Age Group',
    'LKE_ID_max': 'Max Age Group',
    'LKE_ID_safe_mode': 'Mode Age Group',  # Correctly name the mode column
    'Group_<lambda>': 'Group'
}, inplace=True)

print(grouped)


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
grouped.head()

In [ ]:
grouped.reset_index(inplace=True)


In [ ]:
grouped.head()

In [ ]:
grouped.to_csv('Processed data/party_grouped.csv', index=False)


## Checkpoint!

In [ ]:
grouped= pd.read_csv('Processed data/party_grouped.csv')

In [ ]:
import squarify
import seaborn as sb

# Combine 'Party Category' and 'Group', and standardize/sort groups lexicographically
grouped['Combined Group'] = grouped['Party Category'] + ": " + grouped['Group'].apply(lambda x: ', '.join(sorted(x.split(', '))))

# Count the occurrences of each sorted combined group
combined_group_counts = grouped['Combined Group'].value_counts()

# Calculate the 2% threshold of the total counts
threshold = combined_group_counts.sum() * 0.01

# Filter out categories below the threshold
small_groups = combined_group_counts[combined_group_counts < threshold]
other_sum = small_groups.sum()  # Sum of all small group counts
combined_group_counts = combined_group_counts[combined_group_counts >= threshold]  # Keep only groups above threshold

# Add 'Other' category if there are any small groups
if other_sum > 0:
    combined_group_counts['Other'] = other_sum

# Prepare data for plotting
labels = [f"{label}\n({value / 1000:.1f}K)" for label, value in zip(combined_group_counts.index, combined_group_counts.values)]
sizes = combined_group_counts.values
colors = plt.cm.Spectral((sizes - sizes.min()) / (sizes.max() - sizes.min()))

# Plot
plt.figure(figsize=(12, 8))
squarify.plot(sizes=sizes, label=labels, color=sb.color_palette("rocket", len(labels)), text_kwargs={'fontsize': 10, 'color': 'white'}, alpha=0.8, pad=0.25)
plt.title('Treemap of Combined Group Value Counts with "Other" Group for the Lowest 2%')
plt.axis('off')  # Turn off axis
plt.show()


In [ ]:
# Combine 'Party Category' and 'Group', and standardize/sort groups lexicographically
grouped['Combined Group'] = grouped['Party Category'] + ": " + grouped['Group'].apply(lambda x: ', '.join(sorted(x.split(', '))))

# Count the occurrences of each sorted combined group
combined_group_counts = grouped['Combined Group'].value_counts()

# Calculate the 2% threshold of the total counts
threshold = combined_group_counts.sum() * 0.01

# Filter out categories below the threshold
small_groups = combined_group_counts[combined_group_counts < threshold]
other_sum = small_groups.sum()  # Sum of all small group counts
combined_group_counts = combined_group_counts[combined_group_counts >= threshold]  # Keep only groups above threshold

# Add 'Other' category if there are any small groups
if other_sum > 0:
    combined_group_counts['Other'] = other_sum

# Prepare data for the table
table_data = combined_group_counts.reset_index()
table_data.columns = ['Combined Group', 'Counts']

# Print the table
print(table_data.to_string(index=False))

In [ ]:
grouped = grouped.merge(rta[['VKL_NUMMER', 'SEVERE']], on='VKL_NUMMER', how='left')


In [ ]:
grouped['SEVERE'].value_counts()

In [ ]:
rta['SEVERE'].value_counts()

In [ ]:
pa=rta.merge(grouped,on='VKL_NUMMER', how='inner')

In [ ]:
pa.to_csv('Processed data/pa.csv')

In [ ]:
pa= pd.read_csv('Processed data/pa.csv')

## EDA before and after

In [ ]:

# Function to compute proportion of severe cases
def proportion_severe(values):
    return np.mean(values)  # Average of 1s and 0s gives us the proportion of 1s

# Set up the figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), sharey=True)

# Hexbin for data density
hb_density = ax1.hexbin(rta['longitude'], rta['latitude'], gridsize=7, cmap='Reds',
                        bins='log', mincnt=1)  # Logarithmic count of points per hexbin
ax1.set_title('Data Density (Log Scale)')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
fig.colorbar(hb_density, ax=ax1, label='Logarithmic scale of data density')

# Hexbin for proportion of severe cases
hb_severe = ax2.hexbin(rta['longitude'], rta['latitude'], C=rta['SEVERE'],
                       gridsize=7, reduce_C_function=proportion_severe, cmap='Reds',
                       norm=mcolors.LogNorm(vmin=0.01, vmax=1))  # Logarithmic normalization
ax2.set_title('Proportion of SEVERE Cases')
ax2.set_xlabel('Longitude')
fig.colorbar(hb_severe, ax=ax2, label='Logarithmic scale of proportion of SEVERE cases')

plt.show()


In [ ]:
# Create bin centers and spatial index from 'rta' using hexbin
fig, ax = plt.subplots()
hb_rta = ax.hexbin(rta['longitude'], rta['latitude'], gridsize=6, cmap='coolwarm', reduce_C_function=np.mean)
plt.close(fig)  # No need to display this figure

# Retrieve bin centers
bin_centers = hb_rta.get_offsets()

# Build spatial index using cKDTree
tree = cKDTree(bin_centers)

# Find the nearest hexbin index for each data point in RTA
distances, indices = tree.query(rta[['longitude', 'latitude']])

# Assign bin index to each entry in 'rta'
rta['bin_index'] = indices

# Sort bins by latitude to order them from north to south
sorted_indices = np.argsort(-bin_centers[:, 1])
new_ids = np.argsort(sorted_indices)

# Remap bin indices in 'rta' using sorted indices
rta['sorted_bin_index'] = new_ids[rta['bin_index']]

print(rta.head())


In [ ]:
# Create a hexbin plot with Matplotlib and apply logarithmic normalization
fig, ax = plt.subplots()
hb = ax.hexbin(rta['longitude'], rta['latitude'], gridsize=6, cmap='OrRd',
               reduce_C_function=np.mean, norm=LogNorm())
ax.set_title('Hexagonal Binning of Data Points (Log Scale)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
cb = plt.colorbar(hb)
cb.set_label('Logarithmic scale of counts in bin')

# Retrieve bin centers and counts
bin_centers = hb.get_offsets()
counts = hb.get_array()

# Filter out empty bins (where count is zero)
non_empty_bins = counts > 0
filtered_bin_centers = bin_centers[non_empty_bins]

# Build a spatial index using cKDTree for non-empty bins
tree = cKDTree(filtered_bin_centers)

# Find the nearest hexbin for each data point among non-empty bins
distances, indices = tree.query(rta[['longitude', 'latitude']].values)

# Save the bin index and bin center coordinates to the DataFrame
rta['hexbin_id'] = indices
rta['hexbin_center_longitude'] = filtered_bin_centers[indices, 0]
rta['hexbin_center_latitude'] = filtered_bin_centers[indices, 1]

# Sort bins by latitude to order them from north to south
sorted_indices = np.argsort(-filtered_bin_centers[:, 1])  # Sort by negative latitude for descending order
new_ids = np.argsort(sorted_indices)  # This gives a new order to the indices
rta['sorted_hexbin_id'] = new_ids[rta['hexbin_id']]

# Annotate the plot with sorted bin IDs
for i, center in enumerate(filtered_bin_centers):
    sorted_id = new_ids[i]
    ax.text(center[0], center[1], str(sorted_id), color='blue', ha='center', va='center')

plt.legend(['Bin Centers'])
plt.show()


In [ ]:
# Extracting hexbin_id from rta
hexbin_data = rta[['VKL_NUMMER', 'sorted_hexbin_id','hexbin_center_longitude','hexbin_center_latitude']]

# Merging hexbin_id with pa
pa = pd.merge(pa, hexbin_data, on='VKL_NUMMER', how='left')

# Check for any NaN values in the merged DataFrame
missing_values = pa['sorted_hexbin_id'].isna().sum()
if missing_values > 0:
    print(f"There are {missing_values} rows in 'pa' with no corresponding 'hexbin_id'.")



In [ ]:
from scipy.spatial import cKDTree

# Function to create hexbin plot with annotations
def create_hexbin_plot(data, title):
    fig, ax = plt.subplots(figsize=(10,8))
    hb = ax.hexbin(data['longitude'], data['latitude'], gridsize=6, cmap='coolwarm',
                   reduce_C_function=np.mean, norm=LogNorm())
    ax.set_title(title)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    cb = plt.colorbar(hb)
    cb.set_label('Logarithmic scale of counts in bin')

    # Retrieve bin centers and counts
    bin_centers = hb.get_offsets()
    counts = hb.get_array()

    # Filter out empty bins (where count is zero)
    non_empty_bins = counts > 0
    filtered_bin_centers = bin_centers[non_empty_bins]

    # Build a spatial index using cKDTree for non-empty bins
    tree = cKDTree(filtered_bin_centers)

    # Find the nearest hexbin for each data point among non-empty bins
    distances, indices = tree.query(data[['longitude', 'latitude']].values)

    # Save the bin index and bin center coordinates to the DataFrame
    data['hexbin_id'] = indices
    data['hexbin_center_longitude'] = filtered_bin_centers[indices, 0]
    data['hexbin_center_latitude'] = filtered_bin_centers[indices, 1]

    # Sort bins by latitude to order them from north to south
    sorted_indices = np.argsort(-filtered_bin_centers[:, 1])  # Sort by negative latitude for descending order
    new_ids = np.argsort(sorted_indices)  # This gives a new order to the indices
    data['sorted_hexbin_id'] = new_ids[data['hexbin_id']]

    # Annotate the plot with sorted bin IDs
    for i, center in enumerate(filtered_bin_centers):
        sorted_id = new_ids[i]
        ax.text(center[0], center[1], str(sorted_id), color='blue', ha='center', va='center')

    plt.legend(['Bin Centers'])
    plt.show()

# Create hexbin plot with annotations for rta
create_hexbin_plot(rta, 'Hexagonal Binning of Data Points for rta (Log Scale)')

# Create hexbin plot with annotations for pa
create_hexbin_plot(pa, 'Hexagonal Binning of Data Points for pa (Log Scale)')


In [ ]:
pa.columns

In [ ]:
pa['SEVERE']=pa['SEVERE_x']

In [ ]:
pa['sorted_hexbin_id'].value_counts()

In [ ]:
# Function to calculate the ratio of SEVERE=1 to total counts in each bin
def ratio_of_severe(x):
    return np.sum(x) / len(x)

# Modified function to create hexbin plot with custom aggregation
def create_hexbin_plot(data, title):
    fig, ax = plt.subplots()
    hb = ax.hexbin(data['longitude'], data['latitude'], C=data['SEVERE'],
                   gridsize=6, cmap='coolwarm', reduce_C_function=ratio_of_severe, norm=LogNorm())
    ax.set_title(title)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    cb = plt.colorbar(hb)
    cb.set_label('Ratio of SEVERE=1')

    # Retrieve bin centers and counts
    bin_centers = hb.get_offsets()
    counts = hb.get_array()

    # Filter out empty bins (where count is zero)
    non_empty_bins = counts > 0
    filtered_bin_centers = bin_centers[non_empty_bins]

    # Build a spatial index using cKDTree for non-empty bins
    tree = cKDTree(filtered_bin_centers)

    # Find the nearest hexbin for each data point among non-empty bins
    distances, indices = tree.query(data[['longitude', 'latitude']].values)

    # Save the bin index and bin center coordinates to the DataFrame
    data['hexbin_id'] = indices
    data['hexbin_center_longitude'] = filtered_bin_centers[indices, 0]
    data['hexbin_center_latitude'] = filtered_bin_centers[indices, 1]

    # Sort bins by latitude to order them from north to south
    sorted_indices = np.argsort(-filtered_bin_centers[:, 1])  # Sort by negative latitude for descending order
    new_ids = np.argsort(sorted_indices)  # This gives a new order to the indices
    data['sorted_hexbin_id'] = new_ids[data['hexbin_id']]

    # Annotate the plot with sorted bin IDs
    for i, center in enumerate(filtered_bin_centers):
        sorted_id = new_ids[i]
        ax.text(center[0], center[1], str(sorted_id), color='blue', ha='center', va='center')

    plt.legend(['Bin Centers'])
    plt.show()

# Create plots
create_hexbin_plot(rta, 'Hexagonal Binning of SEVERE Ratio for rta (Log Scale)')
create_hexbin_plot(pa, 'Hexagonal Binning of SEVERE Ratio for pa (Log Scale)')


In [ ]:
pa.head()

In [ ]:
pa.shape

In [ ]:
rta.columns

In [ ]:
pa.to_csv('Processed data/pa.csv',index=False)